# Magneto-Stokes Flow in a Shallow Free-Surface Annulus

Cy S. David, Eric W. Hester, Yufan Xu, and Jonathan M. Aurnou

In [ ]:
from magStokesSoln.magStokesSoln import magStokes
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'
import pandas as pd

This notebook demonstrates how to use the `magStokes` package to quickly compute theoretical predictions for magnetohydrodynamically-pumped flow in a shallow free-surface annulus.

## Theory

Consider a layer of conducting fluid of depth $h$ in the annular gap between two cylindrical electrodes of radius $r_i$ and $r_o$ ($r_i<r_o$). A current $I$ runs through the fluid from inner to outer electrode, and the entire annulus is immersed in a vertical, imposed magnetic field $\boldsymbol{B_0} =  -B_0 \boldsymbol{e_z}$. Figure 1(a) shows a schematic of the annular channel with imposed magnetic field and current. Figure 1(b) shows the actual laboratory device with a spiral of blue dye that reveals the swirling flow.

![Diagram.jpg](images/Diagram.jpg)

When the power supply is turned on, the fluid experiences an azimuthal electromagnetic force that drives circulatory flow against viscous drag.

The dimensionless axisymmetric solution for spin-up from rest may be approximated as

$${u}^*_\theta(\tilde{r},\tilde{z},\tilde{t}) =  \left\{\sum_{n=1}^{\infty} \frac{2(\chi+1)}{k_n^2 \gamma} \left[ \frac{\gamma}{k_n \tilde{r}}-A_n I_1\left( \frac{k_n}{ \gamma}\tilde{r}\right) - B_n K_1\left( \frac{k_n}{ \gamma}\tilde{r}\right) \right]\sin (k_n  \tilde{z}) \right\}\times \left[1- \exp\left(-\frac{\pi ^2}{4 \textit{Re}_C}\tilde{t}\right)\right]$$

where  

$ A_n=\frac{\gamma}{k_n \chi}\left[\frac{\chi  K_ 1\left({k_n \chi}/{\gamma }\right)-K_ 1\left({k_n }/{\gamma}\right)}{I_1\left({k_n}/{\gamma }\right) K_1\left({k_n\chi }/{\gamma}\right)-K_1\left({k_n}/{\gamma\\}\right) I_1\left({k_n \chi }/{\gamma}\right)}\right]$,  $B_n=\frac{I_ 1\left({k_n}/{\gamma}\right)-\chi  I_1\left({k_n\chi }/{\gamma}\right)}{\chi  K_1\left({k_n \chi}/{\gamma }\right)-K_1\left({k_n }/{\gamma}\right)} A_n$,  

$k_n=\pi (n-1/2)$,  $\textit{Re}_C = ({h^2/\nu})/({(r_i+r_o)/\mathcal{U}_\text{MS}})$, $\chi = r_i/r_o$, and $\gamma = h/r_o$,

with dimensionless coordinates

$\tilde{r} = r/r_o$,    $\tilde{z}=z/h$,   $t=\tilde{t}(r_i+r_o)/\mathcal{U}_\text{MS}$,  

and velocity scale

$\mathcal{U}_\text{MS} =B_0 h I/(2 \pi  \nu  \rho (r_i+r_o))$.

## Example 1: Plot analytical solution

Create an instance of the `magStokes` class for a flow with 0.4 cm fluid layer depth, 4 cm inner cylinder radius, 18 cm outer cylinder radius, 0.03 Tesla (300 Gauss) magnetic field strength, and 0.08 A applied current:

In [ ]:
flow = magStokes(h=0.004,r_i=0.04,r_o=0.18,B=0.03,Imax=0.08)

Compute the density and viscosity for a saltwater solution with 80 ppt NaCl:H2O. Note that the `.customFluidProps()` method may be used to manually enter viscosity and density for an arbitrary fluid.

In [ ]:
flow.saltwaterProps(S=80)

Print the computed aspect ratios, control Reynolds number, and velocity scale:

In [ ]:
print(flow) 

Use the `.u()` method to compute the solution for an initially-quiescent flow 20 seconds after the power supply (I = 0.08 A) is turned on.

In [ ]:
# Independent variables
t = 20 # seconds
rArr = np.linspace(flow.r_i,flow.r_o,1001)
zArr = np.linspace(flow.h,0,5)

# Compute the steady solution in the shallow limit
# (dashed gray line)
u1DArr = flow.uShallow(r=rArr,z=flow.h)

# Compute and plot 2D solution at each depth in `zArr`
# using the `.u()` method
cmap = matplotlib.cm.get_cmap('Blues')
plt.plot(1e2*rArr,1e2*u1DArr,color='gray',linestyle='--',
         label=f'z={1e2*flow.h:.1f} cm,\n$\\gamma\\to\\infty,\\; t\\to\\infty$')
for z in zArr:
    uArr = flow.u(r=rArr,z=z,t=t)
    plt.plot(1e2*rArr,1e2*uArr,color=cmap(z/flow.h),label=f'z={1e2*z:.1f} cm')
plt.xlabel('r (cm)')
plt.ylabel('u (cm/s)')
plt.legend()
plt.title(f't = {t:.1f} s = {t/flow.Tsp:.2f} $T_{{su}}$')
plt.savefig('images/Soln.jpg',dpi=300)
plt.show()

## Example 2: Compute scales and dimensionless parameters

Read in (dimensional) experimental parameters:

In [ ]:
DimExpParam = pd.read_csv('example_data/example_data.csv')

Compute aspect ratios, control Reyolds number, velocity scale, and time scales from dimensional experimental parameters:

In [ ]:
compData = []

for i in range(DimExpParam.shape[0]):
    run = DimExpParam['RUN'][i]
    h = 1e-2*DimExpParam['Height fluid (cm)'][i]
    r_i = 1e-2*DimExpParam['Inner radius (cm)'][i]
    r_o = 1e-2*DimExpParam['Outer radius (cm)'][i]
    Imax = DimExpParam['Current (A)'][i]
    B = 1e-4*DimExpParam['Magnetic field (Gauss)'][i]
    S = DimExpParam['Salinity (ppt)'][i]

    # Create magStokes instance with specified parameters
    labFlow = magStokes(h=h,r_i=r_i,r_o=r_o,B=B,Imax=Imax)
    # Input salinity. Compute dimensionless numbers and scales
    labFlow.saltwaterProps(S=S)

    # Store dimensionless numbers and scales
    computedParams = [run, labFlow.γ, labFlow.χ, labFlow.ReC, labFlow.U, labFlow.T, labFlow.Tcirc, labFlow.Tsp]
    compData.append(computedParams)

compDataDf = pd.DataFrame(compData,columns=['Case','gamma','chi','ReC','U (m/s)','T (s)','Tcirc (s)','Tsp (s)'])

pd.set_option('display.float_format',  '{:.1e}'.format)
compDataDf

Export computed values:

In [ ]:
compDataDf.to_csv('example_data/example_compData.csv')

## Example 3: Compare theory and lab data

This example requires `skvideo` and ffmpeg.

In [ ]:
import skvideo.io

Import video of laboratory Run A3 along with its metadata:

In [ ]:
fname = 'example_data/RUN_A3.mp4'
vdata = skvideo.io.vread(fname)
metadata = skvideo.io.ffprobe(fname)

Extract frame-rate from the metadata and get the outer radius of channel in pixels (the video is cropped so that the channel just fits within the frame).

In [ ]:
frac_str = metadata['video']['@avg_frame_rate']
num,den=[float(i) for i in frac_str.split('/')]
frame_rate = num/den
r_o_pixel = vdata[0].shape[0]/2

Import parameter values for Run A3 and create a `magStokes` instance called `runA3`.

In [ ]:
DimExpParam = pd.read_csv('example_data/example_data.csv')

RunA3Df = DimExpParam[DimExpParam['RUN']=='RUN A3']
h = 1e-2*RunA3Df['Height fluid (cm)'].values[0]
r_i = 1e-2*RunA3Df['Inner radius (cm)'].values[0]
r_o = 1e-2*RunA3Df['Outer radius (cm)'].values[0]
Imax = RunA3Df['Current (A)'].values[0]
B = 1e-4*RunA3Df['Magnetic field (Gauss)'].values[0]
S = RunA3Df['Salinity (ppt)'].values[0]

# Create magStokes instance with specified parameters
runA3 = magStokes(h=h,r_i=r_i,r_o=r_o,B=B,Imax=Imax)
# Input salinity. Compute dimensionless numbers and scales
runA3.saltwaterProps(S=S)

Compute the predicted position of a dye streak using the `.theta()` method, and plot it against the lab video.

In [ ]:
# Create arrays of radial position
rND = np.linspace(runA3.χ, 1, 1000)
r = runA3.r_o * rND
rPixel = r_o_pixel * rND

fig,ax = plt.subplots(1,3,figsize=(8,3))

for i in range(3):
    # Convert time to frames
    tapprox = runA3.Tsp*3*i
    frame = int(frame_rate*tapprox)
    t = frame/frame_rate

    # Predict position of dye streak using `runA3.theta(...)`
    thetaDye = runA3.theta(r=r,z=runA3.h,t=t,θ0=1.03*np.pi/2)
    thetaShallowDye = runA3.thetaShallow(r=r,z=runA3.h,t=t,θ0=1.03*np.pi/2)
    xDyePixel = r_o_pixel - rPixel*np.cos(thetaDye)
    yDyePixel = r_o_pixel + rPixel*np.sin(thetaDye)
    xDyeShalPixel = r_o_pixel - rPixel*np.cos(thetaShallowDye)
    yDyeShalPixel = r_o_pixel + rPixel*np.sin(thetaShallowDye)

    ax[i].plot(xDyeShalPixel,yDyeShalPixel,'k--',linewidth=1)
    ax[i].plot(xDyePixel,yDyePixel,color='m')
    ax[i].imshow(vdata[frame],origin='upper')
    ax[i].axis('off')
    ax[i].set_title(f'$t = {3*i:.0f}T_{{su}}$')
plt.tight_layout()
#plt.savefig('images/Dye.jpg',dpi=600)
plt.show()

## Example 4: Choosing experimental parameters

For a given fluid layer depth, we can choose the current to apply based on desired spin-up ($T_{su}$) and circulation timescales ($T_{circ}$).

First define a function to plot $T_{su}$ and $T_{circ}$ vs. current:

In [ ]:
def plotTimescales(h,r_i,r_o,B):

    n = 50
    IArr = np.linspace(0.02,0.2,n)
    TspArr = np.zeros(n)
    TcircArr = np.zeros(n)

    for i in range(n):
        Imax = IArr[i]
        flow = magStokes(h=h,r_i=r_i,r_o=r_o,B=B,Imax=Imax)
        flow.saltwaterProps(S=80)
        TspArr[i] = flow.Tsp
        TcircArr[i] = flow.Tcirc

    plt.plot(IArr,3*TspArr,label="$3 T_{su}$")
    plt.plot(IArr,TcircArr,label="$T_{circ}$")
    plt.ylim(0,max(4*TspArr[0],180))
    plt.xlim(IArr[0],IArr[-1])
    plt.xlabel("Current, I (A)",fontsize = 12)
    plt.ylabel("Timescale (s)",fontsize = 12)
    plt.grid()
    plt.legend()
    title = f"Timescales for $h =$ {100*flow.h:.1f} cm\n"
    title += f"$B = $ {flow.B:.2f} T, $r_i = $ {100*flow.r_i:.1f} cm, $r_o = $ {100*flow.r_o:.1f} cm"
    plt.title(title)
    plt.show()

Set depth of fluid, channel radii, and magnetic field. Plot timescales vs. current:

In [ ]:
r_i = 0.04 # m
r_o = 0.18 # m
B = 0.02 # T
h = 0.008 # m

plotTimescales(h,r_i,r_o,B)

The plot above shows that a choice of $I$ = 0.06 A ensures the flow is dominantly viscous ($3T_{su} < T_{circ}$) without making it too slow ($T_{circ}$ is still under 2 minutes).